In [1]:
import os
import pandas as pd
from pandas import DataFrame

pd.set_option("display.max_columns", 50)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.contingency import chi2_contingency
from scipy.stats import normaltest, shapiro
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")

import matplotlib

matplotlib.rcParams["axes.labelsize"] = 9
matplotlib.rcParams["legend.fontsize"] = 9
matplotlib.rcParams["ytick.labelsize"] = 9
matplotlib.rcParams["xtick.labelsize"] = 9

In [2]:
df = pd.read_csv("Telco_customer_churn.csv")
df.head(4)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved


In [2]:
import pandas as pd
import numpy as np
import datetime

# Load your data
df = pd.read_csv('Telco_customer_churn.csv')

# Generate random start dates within a broader range
np.random.seed(0)
start_date = pd.to_datetime('2018-01-01')
end_date = pd.to_datetime('2024-12-30')  # Ensure this extends past the second price increase date
df['Start_Date'] = start_date + (end_date - start_date) * np.random.rand(len(df))

# Calculate End Dates based on 'Tenure Months'
df['End_Date'] = df['Start_Date'] + pd.to_timedelta(df['Tenure Months'] * 30, unit='D')

# Define the price increase dates
first_increase_date = datetime.datetime(2023, 12, 1)
second_increase_date = datetime.datetime(2024, 11, 1)

# Create binary columns for price increases
df['Price_Increase_Dec_2023'] = df['Start_Date'].apply(lambda x: 1 if x < first_increase_date else 0)
df['Price_Increase_Nov_2024'] = df['Start_Date'].apply(lambda x: 1 if x < second_increase_date else 0)

# Correct the logic for Price_Increase_Nov_2024
df['Price_Increase_Nov_2024'] = df['Start_Date'].apply(lambda x: 1 if first_increase_date <= x < second_increase_date else 0)

# Create tenure after price increase columns
df['Tenure_After_First_Increase'] = ((df['End_Date'] - first_increase_date).dt.days / 30).clip(lower=0)
df['Tenure_After_Second_Increase'] = ((df['End_Date'] - second_increase_date).dt.days / 30).clip(lower=0)

# Display the first few rows to verify
print(df[['CustomerID', 'Start_Date', 'End_Date', 'Price_Increase_Dec_2023', 'Price_Increase_Nov_2024', 'Tenure_After_First_Increase', 'Tenure_After_Second_Increase']].head())

# Verify the value counts
print("Price_Increase_Dec_2023 value counts:")
print(df['Price_Increase_Dec_2023'].value_counts())

print("Price_Increase_Nov_2024 value counts:")
print(df['Price_Increase_Nov_2024'].value_counts())


   CustomerID                    Start_Date                      End_Date   
0  3668-QPYBK 2021-11-03 05:14:38.618964800 2022-01-02 05:14:38.618964800  \
1  9237-HQITU 2023-01-02 07:24:43.005444352 2023-03-03 07:24:43.005444352   
2  9305-CDSKC 2022-03-21 01:27:00.794567536 2022-11-16 01:27:00.794567536   
3  7892-POOKP 2021-10-24 04:14:12.412930976 2024-02-11 04:14:12.412930976   
4  0280-XJGEX 2020-12-18 10:30:44.263661888 2024-12-27 10:30:44.263661888   

   Price_Increase_Dec_2023  Price_Increase_Nov_2024   
0                        1                        0  \
1                        1                        0   
2                        1                        0   
3                        1                        0   
4                        1                        0   

   Tenure_After_First_Increase  Tenure_After_Second_Increase  
0                     0.000000                      0.000000  
1                     0.000000                      0.000000  
2                

In [3]:
df.head(70)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Churn Value,Churn Score,CLTV,Churn Reason,Start_Date,End_Date,Price_Increase_Dec_2023,Price_Increase_Nov_2024,Tenure_After_First_Increase,Tenure_After_Second_Increase
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,1,86,3239,Competitor made better offer,2021-11-03 05:14:38.618964800,2022-01-02 05:14:38.618964800,1,0,0.000000,0.000000
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,1,67,2701,Moved,2023-01-02 07:24:43.005444352,2023-03-03 07:24:43.005444352,1,0,0.000000,0.000000
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,1,86,5372,Moved,2022-03-21 01:27:00.794567536,2022-11-16 01:27:00.794567536,1,0,0.000000,0.000000
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,1,84,5003,Moved,2021-10-24 04:14:12.412930976,2024-02-11 04:14:12.412930976,1,0,2.400000,0.000000
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,1,89,5340,Competitor had better devices,2020-12-18 10:30:44.263661888,2024-12-27 10:30:44.263661888,1,0,13.066667,1.866667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,6323-AYBRX,1,United States,California,Ontario,91762,"34.057256, -117.667677",34.057256,-117.667677,Male,...,1,83,5084,Price too high,2020-07-31 02:13:38.873749152,2025-06-05 02:13:38.873749152,1,0,18.400000,7.200000
66,4385-GZQXV,1,United States,California,Ontario,91764,"34.074087, -117.605618",34.074087,-117.605618,Female,...,1,81,3810,Product dissatisfaction,2023-09-29 15:18:17.475391360,2025-01-21 15:18:17.475391360,1,0,13.900000,2.700000
67,6366-ZGQGL,1,United States,California,Pomona,91766,"34.042286, -117.756106",34.042286,-117.756106,Male,...,1,73,5087,Service dissatisfaction,2018-09-06 02:15:00.833869860,2018-10-06 02:15:00.833869860,1,0,0.000000,0.000000
68,5940-AHUHD,1,United States,California,Pomona,91768,"34.067932, -117.785168",34.067932,-117.785168,Male,...,1,69,3338,Lack of self-service on Website,2023-11-11 22:46:54.220175968,2023-12-11 22:46:54.220175968,1,0,0.333333,0.000000
